In [69]:
import os
import pandas as pd
import numpy as np
import collections

In [70]:
grch37 = "/data/database/GRCh37/gencode.v19.annotation.gff3"
grch38 = "/data/database/GRCh38/GENCODE/gencode.v27.annotation.gff3"

In [78]:
def generate_dict(a_list, b_list):
    raw_dict, result_dict = {}, {}
    assert len(a_list) == len(b_list)
    for i in range(len(a_list)):
        raw_dict[a_list[i]] = raw_dict.get(a_list[i], []) + [b_list[i]]
    for a, b in raw_dict.items():
        result_dict[a] = ";".join(sorted(list(set(b))))
    return result_dict

def generate_grch_dict(gff3):
    df = pd.read_table(gff3, sep="\t", comment="#", header=None)
    df_gene = df[df.iloc[:, 2].str.contains("gene")]
    df_gene["Ensembl_ID"] = df_gene.iloc[:, 8].str.split("ID=").str[1].str.split(".").str[0]
    df_gene["Symbol"] = df_gene.iloc[:, 8].str.split("gene_name=").str[1].str.split(";").str[0]
    grch_dict = generate_dict(df_gene["Symbol"].tolist(), df_gene["Ensembl_ID"].tolist())
    return grch_dict

In [79]:
grch37_dict = generate_grch_dict(grch37) # , "GRCh37"
grch38_dict = generate_grch_dict(grch38) # , "GRCh38"

/home/galaxy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/galaxy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [80]:
df_37 = pd.DataFrame.from_dict(grch37_dict, orient="index", columns=['Ensembl_ID_37'])
df_38 = pd.DataFrame.from_dict(grch38_dict, orient="index", columns=['Ensembl_ID_38'])
df_37["Symbol"] = df_37.index
df_38["Symbol"] = df_38.index
df_final = df_37.merge(df_38, how="outer")
# print(df_final.head())
df_final.to_csv("/data/database/GRCh37/ensembl37_to_ensembl38.txt", sep="\t", index=False)